In [2]:
#%pylab inline

import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_datasets as tfds
#  Important !!!! path to shared tensorflow dataset
#data_dir='/storage/scratch/mhuertas/data/sfh/tensorflow_datasets/eagle'

data_dir ='/scratch/mhuertas/tensorflow_datasets/eagle'

In [3]:
from sfh.datasets.eagle import eagle

In [5]:
dset = tfds.load('eagle', split='train', data_dir=data_dir)

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /scratch/mhuertas/tensorflow_datasets/eagle/eagle/3.0.0.incompleteNQVH9K/eagle-train.tfrecord*...:  …

Dataset eagle downloaded and prepared to /scratch/mhuertas/tensorflow_datasets/eagle/eagle/3.0.0. Subsequent calls will reuse this data.


2022-04-13 12:30:37.584601: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [28]:
import numpy as np
# Reordering the wavelenghts
#wl = np.loadtxt('/storage/scratch/mhuertas/data/sfh/tensorflow_datasets/tng100/downloads/manual/wl.csv')
wl = np.loadtxt('/scratch/mhuertas/tensorflow_datasets/eagle/downloads/manual/wl.csv')
inds = np.argsort(wl)
print(inds)

[ 22  20  93 105  91  23  21 106  92 130 107  24 131 108 132  81  71  90
  79  45 109  82 133 117 110   3   8   7  84 134  80  68 118  76  85 111
  13  61  42 112 119 135  83  88  94  19 113 120 136  63  70  44  87 121
  15  18  12  10  78  34  69  62 114  86  43 122  14  11  77 137 115 123
   9  65 129  46  72  35 124  89 138  17 116  64  99  16  98 128  52 100
  56  36 101  53   1 102 126  96  54 103  37 104  50  57 125  95   0  51
  38  97  33   2  55 127  39 139   4  40  25   5  41  26 140   6  27  28
  30 141 142  31  48  32  60  49  29  58  47  59  73  74  66  75  67]


In [9]:
""""Keras model implementing PixelCNN."""

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras
from tensorflow.keras import layers
import sys
import time
tfd = tfp.distributions
tfb = tfp.bijectors
tfkl=keras.layers

def generate_model(n_timesteps, n_filters, *, n_channels=1, n_components=2, kernel_size=3,
                   n_dilations=5, list_of_dilation_rates=None,
                   list_of_filters=None):
    """Generate the PixelCNN Keras model.

    Parameters
    ----------
    n_timesteps : int
        Number of time steps.
    n_filters : int
        Number of filters.
    n_channels : int, default 1
        Number of channels in the dataset
    n_components : int, default 2
        Number of components in the Gaussian mixture distribution.
    kernel_size : int, default 3
        Size of the convolution kernel.
    n_dilations : int, default 5
        Number of dilated convolutions to do. For each convolution, the
        dilation rate is 2**idx+1 and the number of filters is 2**idx+4.
    list_of_dilation_rates : list of int or None, default None
        List of the dilation rates to use in the dilated convolutions. If not
        None, the n_dilations is not used and filters must be given with the
        same size.
    list_of_filters : list of int or None, default None
        List of the filter number for each of the dilated convolutions. Must be
        of the same size as list_of_dilation_rates

    Returns
    -------
    Keras model

    """
    # Shape of the distribution
    event_shape = [1]
    # Compute how many parameters this distribution requires
    params_size = 2
    #print(params_size)

    
    input_sfh = keras.layers.Input(shape=(n_timesteps,1))
    input_sed = keras.layers.Input(shape=(n_filters,1))
    
    # Compress the SED and return some channels
    sed_net = tf.keras.Sequential([
        tfkl.Input(shape=(125, 1)),
        tfkl.Conv1D(16, 3, strides=2, padding='same', activation='relu'),
        tfkl.Conv1D(32, 3, strides=2, padding='same', activation='relu'),
        tfkl.Conv1D(64, 3, strides=2, padding='same', activation='relu'),
        tfkl.Conv1D(64, 3, strides=1, padding='same', activation='relu'),
        tfkl.Flatten(),
        tfkl.Dense(128, activation='relu'),
        tfkl.Dense(8, activation='softplus'),
        tfkl.Lambda(lambda x: tf.tile(tf.reshape(x,[-1,1,8]), [1,100,1]))
        ])
    
    merged = keras.layers.Concatenate(axis=-1)([input_sfh, 
                                                sed_net(input_sed)])
    
    
    # Shift and cut
    net = keras.layers.Lambda(
            lambda x: tf.pad(x, paddings=tf.constant([[0, 0], [1, 0], [0, 0]]))
        )(merged)
    
    net=keras.layers.Lambda(
            lambda x: x[:, :-1, :]
        )(net)
    

    net=keras.layers.Conv1D(
            filters=16,
            kernel_size=kernel_size,
            dilation_rate=1,
            padding='causal',
            activation='relu'
        )(net)

    if list_of_dilation_rates is None:
        list_of_dilation_rates = [2**(i+1) for i in range(n_dilations)]
        list_of_filters = [2**(i+4) for i in range(n_dilations)]
    elif len(list_of_filters) != len(list_of_dilation_rates):
        raise ValueError(
            "filters and list_of_dilation_rates must have the same length")

    for dilation_rate, nb_filters in zip(list_of_dilation_rates,
                                         list_of_filters):
        net = keras.layers.Conv1D(
                filters=nb_filters,
                kernel_size=kernel_size,
                dilation_rate=dilation_rate,
                padding='causal',
                activation='relu')(net)
    
    net = keras.layers.Dense(2)(net)
    
    net = tfp.layers.DistributionLambda(
                    make_distribution_fn=lambda t: tfd.Beta(
                          concentration1=tf.math.softplus(t[..., 0])+1e-3,
                          concentration0=tf.math.softplus(t[..., 1])+1e-3)
                    )(net)
    
    pixel_cnn = keras.models.Model(inputs=[input_sfh, input_sed],
                                  outputs=net)

    # Use the negative log-likelihood as loss function.
    def negloglik(y, q):
        return tf.reduce_sum(-q.log_prob(y[...,0]), -1)
    
    opt = tf.keras.optimizers.Adam(learning_rate=0.0002)
    pixel_cnn.compile(loss=negloglik, optimizer=opt)

    return pixel_cnn

In [29]:
def preprocessing(example):
    return tf.reshape(example['SFR_Max'],(-1,100,1)), \
           tf.reshape(example['SFR_Max'],(-1,100,1))

def preprocessing_wmass(example):
    mass = example['Mstar'][:,0]
    mass_half = example['Mstar_Half'][:,0]
    tiler = tf.constant([100])
    mass = tf.reshape(tf.tile(mass, tiler),(-1,100,1))
    mass_half = tf.reshape(tf.tile(mass_half, tiler),(-1,100,1))
    sfr = tf.math.add(tf.reshape(example['SFR_Max'],(-1,100,1)), 1e-5)
    res = tf.concat([sfr, mass, mass_half], axis=2)
    return res, res

def preprocessing_wmass_atan(example):
    mass = example['Mstar'][:,0]
    #mass_half = example['Mstar_Half'][:,0]
    #sed = (tf.gather(example['sed'],inds, axis=1) + 20.70243)/2.0466275
    sed = example['sed']
    tiler = tf.constant([100])
    mass = tf.reshape(tf.tile(mass, tiler),(-1,100,1))
    #mass_half = tf.reshape(tf.tile(mass_half, tiler),(-1,100,1))
    sfr = tf.math.tanh(tf.math.asinh(tf.reshape(example['SFR_Max'],(-1,100,1))/40) + 1e-3 + 0.005*tf.math.softplus(tf.random.normal(shape=[64,100,1])))
    res = tf.concat([sfr], axis=2) #  mass, mass_half
    return (res, sed), res

def input_fn(mode='train', batch_size=64, 
             dataset_name='tng100', data_dir=None,
             include_mass=True, arctan=True):
    """
    mode: 'train' or 'test'
    """
    keys = ['sed','Mstar', 'SFR_Max', 'mass_quantiles', 'sed', 'time']
    if mode == 'train':
        dataset = tfds.load(dataset_name, split='train[:90%]', data_dir=data_dir)
        dataset = dataset.map(lambda x: {k:x[k] for k in keys})
        dataset = dataset.repeat()
        dataset = dataset.shuffle(10000)
    else:
        dataset = tfds.load(dataset_name, split='train[90%:]', data_dir=data_dir)
        dataset = dataset.map(lambda x: {k:x[k] for k in keys}) #dataset = dataset.repeat()
        
    dataset = dataset.batch(batch_size, drop_remainder=True)
    if include_mass and arctan:
        dataset = dataset.map(preprocessing_wmass_atan) # Apply data preprocessing
    elif include_mass:
        dataset = dataset.map(preprocessing_wmass)
    else : 
        dataset = dataset.map(preprocessing)
    dataset = dataset.prefetch(-1)       # fetch next batches while training current one (-1 for autotune)
    return dataset

In [31]:
batch_size = 64
epochs = 10

dtrain = input_fn(mode='train', batch_size=batch_size, dataset_name='eagle',data_dir=data_dir)
#dval = input_fn(mode='val', batch_size=batch_size, dataset_name='eagle',data_dir=data_dir)

In [27]:
#import matplotlib.pyplt as plt
print("Train",len(dset))

for example in dset.take(1):
    sed = (tf.gather(example['sed'],inds, axis=1) + 20.70243)/2.0466275

Train 7314


InvalidArgumentError: Shape must be at least rank 2 but is rank 1 [Op:GatherV2]

--- Logging error ---
Traceback (most recent call last):
  File "/opt/python/python3.8/lib/python3.8/site-packages/ipykernel/kernelbase.py", line 461, in dispatch_queue
    await self.process_one()
  File "/opt/python/python3.8/lib/python3.8/site-packages/ipykernel/kernelbase.py", line 450, in process_one
    await dispatch(*args)
TypeError: object NoneType can't be used in 'await' expression

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.8/logging/__init__.py", line 1085, in emit
    self.flush()
  File "/usr/lib/python3.8/logging/__init__.py", line 1065, in flush
    self.stream.flush()
OSError: [Errno 5] Input/output error
Call stack:
  File "/usr/lib/python3.8/runpy.py", line 192, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/python/python3.8/lib/python3.8/site-packages/ipykern

In [32]:
pixel_cnn = generate_model(100,125)

pixel_cnn.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 125, 1)]     0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 100, 1)]     0           []                               
                                                                                                  
 sequential_1 (Sequential)      (None, 100, 8)       152424      ['input_5[0][0]']                
                                                                                                  
 concatenate_1 (Concatenate)    (None, 100, 9)       0           ['input_4[0][0]',                
                                                                  'sequential_1[0][0]']     

In [ ]:
hist = pixel_cnn.fit(dtrain, 
                     epochs=epochs,
                     steps_per_epoch=1000)

Epoch 1/10


1000/1000 [==============================] - 260s 14ms/step - loss: -229.2484
Epoch 2/10
1000/1000 [==============================] - 14s 14ms/step - loss: -265.5918
Epoch 3/10
1000/1000 [==============================] - 15s 15ms/step - loss: -301.1838
Epoch 4/10
1000/1000 [==============================] - 15s 15ms/step - loss: -313.8039
Epoch 5/10
1000/1000 [==============================] - 14s 14ms/step - loss: -318.1458
Epoch 6/10
1000/1000 [==============================] - 14s 14ms/step - loss: -320.5878
Epoch 7/10
 280/1000 [=======>......................] - ETA: 10s - loss: -321.4611